In [1]:
import matplotlib.pyplot as plt
#     Dependencies    #
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import pad_sequences
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from plot_keras_history import plot_history
%matplotlib notebook
#       Seeding      #
tf.random.set_seed(42)
np.random.seed(42)

In [296]:
data=pd.read_csv('train.csv')
data.fillna(0,inplace=True)
data.head(5)

,Time,SOP,label
0,4.0,0.0,0
1,3.0,0.0,0
2,4.0,0.0,0
3,5.0,0.0,0
4,6.0,0.0,0


In [297]:
X=data['SOP']
y=data['label']
y.value_counts()

label
0    35941
2     8760
3     4800
1     4440
Name: count, dtype: int64

In [298]:
count=[]
temp=[]
for index , value in y.items():
    if value==0:
        temp.append(0)
        if (index+1)<len(y) and y[index+1]==1 or 2 or 3:
            count.append(temp)
            temp=[]
print(count[0])      

[0]


In [299]:
plt.plot(X,color='blue')
plt.plot(y,color='red',linestyle='dashed')
plt.xlabel('X-value')
plt.ylabel('Labels')
plt.show()

In [300]:
# Extracting sequences from the dataset
sequences=[]
labels=[]
temp=[]

for index,value in y.items():
    if value==1:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(1)
            temp=[]
    if value==0:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==1) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(0)
            temp=[]
    if value==2:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==1) or ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(2)
            temp=[]
    if value==3:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==1):
            sequences.append(temp)
            labels.append(3)
            temp=[]

In [301]:
one=0
zero=0
two=0
three=0
for value in labels:
    if value==0:
        zero+=1
    if value==1:
        one+=1
    if value==2:
        two+=1
    if value==3:
        three+=1
print(zero,"\n",one,"\n",two,"\n",three)

120 
 60 
 60 
 60


In [302]:
sequences[0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [303]:
# Padding and reshaping
X_train=pad_sequences(sequences,padding='post',truncating='post',dtype=float,maxlen=50)

In [304]:
# Reshaping Labels
y_train=np.array(labels)

In [305]:
sampler=RandomUnderSampler()
X_train,y_train=sampler.fit_resample(X_train,y_train)

In [306]:
# Reshaping Features
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train=y_train.reshape(-1,1)

In [307]:
unique_values, counts = np.unique(y_train, return_counts=True)

# Print the unique values and their respective counts
for value, count in zip(unique_values, counts):
    print(f"Value {value}: Count {count}")

Value 0: Count 60
Value 1: Count 60
Value 2: Count 60
Value 3: Count 60


In [308]:
# It is because Our each  sequence has max laength of 50 and in output we want to predict a label
Tx=50
Ty=1
repeator = tf.keras.layers.RepeatVector(Tx)
concatenator = tf.keras.layers.Concatenate(axis=-1)
densor1 =tf.keras.layers. Dense(10, activation = "tanh")
densor2 = tf.keras.layers.Dense(1, activation = "relu")
activator = tf.keras.layers.Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor =tf.keras.layers. Dot(axes = 1)

In [309]:
def one_step_attention(a,s_prev):
    # We done this to change s_prev to shape of(m,Tx,n_s) for cocatination with a, because concatenation requires all dimension same accept concat axis in this 
    # case is last one so a and s_prev have same first two dims (m,Tx) but last one is changed
    s_prev=repeator(s_prev)
    # We will here concatenate a and s_prev
    concat=concatenator([a,s_prev])
    # here i will calculate energies with 2 dense layers
    e=densor1(concat)
    energies=densor2(e)
    # we know alpha is softmax of this energy
    alpha=activator(energies)
    # to calculate context vector we take dot product of alpha and a
    context_vector=dotor([alpha,a])
    return context_vector

In [310]:
n_a = 64 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 128 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.
post_activation_LSTM_cell = tf.keras.layers.LSTM(n_s, return_state = True) # Please do not modify this global variable.
output_layer = tf.keras.layers.Dense(4, activation='softmax')

In [311]:
def modelf(Tx,Ty,n_a, n_s):
    X=tf.keras.layers.Input(shape=(X_train.shape[1],1)) # because embedding layer only demands the sequence length if i give full shape like (m,Tx) the
    # ouput of embedding layer will be 4D which can not be fed into BILSTM
    # hidden state for post LSTM
    s0 = tf.keras.layers.Input(shape=(n_s,), name='s0')
    # cell state for post lstm
    # because we know From CampusX that shape of hidden and cell state of lstm are equal
    c0 = tf.keras.layers.Input(shape=(n_s,), name='c0')
    s=s0
    c=c0
    outputs = []

    a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_a,return_sequences=True),input_shape=(X_train.shape[1],1))(X)

    for t in range(Ty):
        context=one_step_attention(a,s)
        _,s,c=post_activation_LSTM_cell(context,initial_state = [s,c] )
        out = output_layer(s)
        outputs.append(out)



    print(outputs)
    model=tf.keras.models.Model(inputs=[X,s0,c0],outputs=outputs)

    return model

In [312]:
model = modelf(Tx, Ty, n_a, n_s)

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'dense_30')>]


In [313]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001,beta_1=0.9,beta_2=0.999)#0.00001
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='min', verbose=1,start_from_epoch=5)

In [314]:
# For Training Set
m=X_train.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [315]:
model_history=model.fit([X_train, s0, c0], y_train, epochs=1000, batch_size=100,callbacks=[early_stopping])

Epoch 1/1000
3/3 [==============================] - 7s 66ms/step - loss: 1.3868 - accuracy: 0.4333
Epoch 2/1000
3/3 [==============================] - 0s 74ms/step - loss: 1.3868 - accuracy: 0.2500
Epoch 3/1000
3/3 [==============================] - 0s 117ms/step - loss: 1.3867 - accuracy: 0.0042
Epoch 4/1000
3/3 [==============================] - 0s 72ms/step - loss: 1.3867 - accuracy: 0.0000e+00
Epoch 5/1000
3/3 [==============================] - 0s 62ms/step - loss: 1.3867 - accuracy: 0.0000e+00
Epoch 6/1000
3/3 [==============================] - 0s 60ms/step - loss: 1.3866 - accuracy: 0.0000e+00
Epoch 7/1000
3/3 [==============================] - 0s 78ms/step - loss: 1.3866 - accuracy: 0.0042
Epoch 8/1000
3/3 [==============================] - 0s 71ms/step - loss: 1.3866 - accuracy: 0.0125
Epoch 9/1000
3/3 [==============================] - 0s 101ms/step - loss: 1.3866 - accuracy: 0.0167
Epoch 10/1000
3/3 [==============================] - 0s 73ms/step - loss: 1.3865 - accuracy: 0.

In [228]:
plot_history(model_history,show_standard_deviation=False,show_average=True)

<IPython.core.display.Javascript object>

(<Figure size 1000x500 with 2 Axes>,
 array([<Axes: title={'center': 'Loss'}, xlabel='Epochs', ylabel='Loss'>,
        <Axes: title={'center': 'Accuracy'}, xlabel='Epochs', ylabel='Accuracy'>],
       dtype=object))

In [329]:
#model.save_weights("MODEL-WITH-ATTENTION-CLASSIFICATION.h5")
model.load_weights("MODEL-WITH-ATTENTION-CLASSIFICATION.h5")

**TESTING THE MODEL:**

In [330]:
test_data=pd.read_csv("test.csv")
test_data.fillna(0,inplace=True)
test_data.head(5)

,Time,SOP,label
0,2.0,0.0,0
1,3.0,0.0,0
2,4.0,0.0,0
3,5.0,0.0,0
4,6.0,0.0,0


In [331]:
nan_mask = test_data['SOP'].isna()

# Get the index of NaN values
nan_index = test_data.index[nan_mask]

# Print the index of rows with NaN values
print(nan_index)

Index([], dtype='int64')


In [332]:
X=test_data['SOP']
y=test_data['label']

In [333]:
# Extracting sequences from the dataset
sequences=[]
labels=[]
temp=[]

for index,value in y.items():
    if value==1:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(1)
            temp=[]
    if value==0:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==1) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(0)
            temp=[]
    if value==2:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==1) or ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==3):
            sequences.append(temp)
            labels.append(2)
            temp=[]
    if value==3:
        temp.append(X[index])
        if ((index+1)<len(y) and y[index+1]==0) or ((index+1)<len(y) and y[index+1]==2) or ((index+1)<len(y) and y[index+1]==1):
            sequences.append(temp)
            labels.append(3)
            temp=[]

In [334]:
one=0
zero=0
two=0
three=0
for value in labels:
    if value==0:
        zero+=1
    if value==1:
        one+=1
    if value==2:
        two+=1
    if value==3:
        three+=1
print(zero,"\n",one,"\n",two,"\n",three)

76 
 38 
 38 
 38


In [335]:
# Padding and reshaping
X_train=pad_sequences(sequences,padding='post',truncating='post',dtype=float,maxlen=50)

In [336]:
# Reshaping Labels
y_train = np.array(labels)
sampler = RandomUnderSampler()
X_train, y_train = sampler.fit_resample(X_train, y_train)
# Reshaping Features
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train = y_train.reshape(-1, 1)
print(y_train.shape)

(152, 1)


In [337]:
unique_values, counts = np.unique(y_train, return_counts=True)

# Print the unique values and their respective counts
for value, count in zip(unique_values, counts):
    print(f"Value {value}: Count {count}")

Value 0: Count 38
Value 1: Count 38
Value 2: Count 38
Value 3: Count 38


In [338]:
# For Testing Set
m=X_train.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [339]:
predictions=model.predict([X_train,s0,c0])
predictions=np.round(predictions)

5/5 [==============================] - 0s 13ms/step


In [340]:
predictions=np.argmax(predictions, axis=1)

In [341]:
predictions=np.array(predictions)
predictions=predictions.reshape(-1,1)
print(predictions)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]]


In [342]:
report = classification_report(y_train,predictions)
print(report)

              precision    recall  f1-score   support

           0       0.44      1.00      0.61        38
           1       1.00      0.74      0.85        38
           2       0.00      0.00      0.00        38
           3       1.00      1.00      1.00        38

    accuracy                           0.68       152
   macro avg       0.61      0.68      0.62       152
weighted avg       0.61      0.68      0.62       152


D:\ANACONDA\envs\Tensorflow2.12\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\envs\Tensorflow2.12\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\envs\Tensorflow2.12\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
